<h3>Imports</h3>

In [4]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import gmplot
from IPython.display import IFrame
import tensorflow as tf

Read datasets

In [2]:
df = pd.read_csv('google-smartphone-decimeter-challenge/train/2020-05-14-US-MTV-1/Pixel4/Pixel4_derived.csv', nrows=1)
derived = df.drop(0)
df = pd.read_csv('google-smartphone-decimeter-challenge/train/2020-05-14-US-MTV-1/Pixel4/ground_truth.csv', nrows=1)
ground = df.drop(0)

for dirname, _, filenames in os.walk('google-smartphone-decimeter-challenge/train'):
    for filename in filenames:
        if 'derived.csv' in filename:
            r = os.path.join(dirname, filename)
            derived = derived.append(pd.read_csv(os.path.join(dirname, filename)))
        elif 'ground_truth.csv' in filename:
            ground = ground.append(pd.read_csv(os.path.join(dirname, filename)))

bsln_trn = pd.read_csv('google-smartphone-decimeter-challenge/baseline_locations_train.csv')
bsln_tst = pd.read_csv('google-smartphone-decimeter-challenge/baseline_locations_test.csv')
target = ground.merge(bsln_trn, how='inner', on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], suffixes=('_grnd', '_bsln'))


FileNotFoundError: [Errno 2] No such file or directory: 'google-smartphone-decimeter-challenge/train/2020-05-14-US-MTV-1/Pixel4/Pixel4_derived.csv'

In [ ]:
target['latDiff'] = target['latDeg_grnd']-target['latDeg_bsln']
target['lngDiff'] = target['lngDeg_grnd']-target['lngDeg_bsln']

In [ ]:
target[['latDiff', 'lngDiff']].describe()

In [ ]:


derived = pd.read_csv('../input/google-smartphone-decimeter-challenge/train/2020-05-14-US-MTV-1/Pixel4/Pixel4_derived.csv')
ground = pd.read_csv('../input/google-smartphone-decimeter-challenge/train/2020-05-14-US-MTV-1/Pixel4/ground_truth.csv')
derived.set_index(['millisSinceGpsEpoch', 'svid'], inplace=True)

In [ ]:
def plotSatelite(ax, x):
    ax.scatter(x['xSatPosM'], x['ySatPosM'], x['zSatPosM'], c='r',s=100)
    ax.quiver(x['xSatPosM'], x['ySatPosM'], x['zSatPosM'], x['xSatVelMps'], x['ySatVelMps'], x['zSatVelMps'], length=(x['xSatVelMps']**2+x['ySatVelMps']**2+x['zSatVelMps']**2)**0.5)

In [ ]:
def plotSphere(ax, r=6731000, center=(0,0,0), hRange=(0, 2 * np.pi), vRange=(0, np.pi)):
    theta = np.array([np.linspace(hRange[0], hRange[1], 50)])
    theta = np.ones_like(theta).T @ theta
    phi = np.array([np.linspace(vRange[0], vRange[1], 50)])
    phi = np.ones_like(phi).T @ phi
    phi = phi.T
    
    xx = r * np.sin(phi) * np.cos(theta) + center[0]
    yy = r * np.sin(phi) * np.sin(theta) + center[1]
    zz = r * np.cos(phi) + center[2]
    
    ax.plot_surface(xx, yy,zz)
    plt.show()

In [ ]:

ax = plt.figure().add_subplot(projection='3d')
timestamp = derived.iloc[0].name[0]
satelites = satelites = derived.loc[timestamp]
satelites.apply(lambda sat: plotSatelite(ax, sat), axis=1)
plt.title("collectionName: {}  || Phone: {} ||  timestamp: {} ms".format(satelites.iloc[0]['collectionName'], satelites.iloc[0]['phoneName'], timestamp))
plotSphere(ax)
ax.scatter(0,0,0, c='g',s=200)


plt.show()

In [ ]:
!pip install gmplot

In [ ]:


target = ground.groupby('millisSinceGpsEpoch').agg(np.average)[['latDeg', 'lngDeg']]
y = np.array(target)

def plotPoints(y_test, pred):
    # import gmplot package
    
    # GoogleMapPlotter return Map object
    # Pass the center latitude and
    # center longitude
    gmap1 = gmplot.GoogleMapPlotter(y[0,0], y[0,1], 12 )
    gmap1.plot(y_test[:,0],y_test[:,1],'cornflowerblue', edge_width=5)
    #gmap1.scatter(y_test[:,0],y_test[:,1],'cornflowerblue', size=3)
    gmap1.plot(pred[:,0],pred[:,1],'red', edge_width=1)
    #gmap1.scatter(pred[:,0],pred[:,1],'red', size=3)
    # Pass the absolute path
    gmap1.draw( "map.html" )

In [ ]:
plotPoints(y, None)

In [ ]:
derived['signalType'] = derived['signalType'].astype('category')
derived['signalType'] = derived['signalType'].cat.codes
X = derived.drop(['collectionName', 'phoneName'], axis=1)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessingial([
    normalizer,
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(3, activation='relu'),
    tf.keras.layers.Dense(5, activation='relu'),
    tf.keras.layers.Dense(2)
])

model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
model.summary()

In [ ]:
target

In [ ]:
X = pd.merge(X, target, how='left', on='millisSinceGpsEpoch')

In [ ]:
y = np.array(X[['latDeg', 'lngDeg']])

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(np.array(X.drop(['latDeg', 'lngDeg'],axis=1)),y, test_size=0.2)

In [ ]:
model.summary()

In [ ]:
x_train[0].shape

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
model.fit(x_train, y_train, epochs=1, validation_data=(x_train, y_train))

In [ ]:
pred = model.predict(x_test)

In [ ]:
y_test.shape

In [ ]:
plotPoints(y, c)
IFrame(src='./map.html', width=700, height=600)

In [ ]:
ground.head(1)

In [ ]:
c = bsln_trn[bsln_trn['collectionName']=='2020-05-14-US-MTV-1'][['latDeg','lngDeg']]
c = np.array(c)

In [ ]:
IFrame(src='./map.html', width=700, height=600)